## Практическая работа

В этой работе вам нужно будет подготовить финальный нотбук по задаче предсказания стоимости поддержанного автомобиля, которую мы решали на протяжении нескольких модулей.
<br>
*Обязательные задачи* помогут проверить, что вы действительно усвоили материал. Если у вас всё получилось, можете переходить к следующей теме.
<br>
Удачи!

Цели практической работы:

1.   Потренироваться в формировании понятного и структурированного ноутбука по результатам исследования.
2.   Отработать выбор наилучшей предсказательной модели с помощью кросс-валидации.
3.   На основе результатов метрик научиться делать выводы о переобучении/недообучении модели.
4.   Научиться сохранять обученные модели в формате `pickle`.




Что оценивается:

*   Ноутбук состоит из блоков: каждый блок соответствует определённой фазе работы с задачей.
*   Блоки, которые состоят из нескольких этапов, декомпозированы на подблоки (например, в `Data Preparation` может быть несколько подблоков: `Data cleaning`, `Feature Selection` и тому подобное).
*   Код написан в едином стиле и не нарушает PEP-8.
*   Код покрыт комментариями.
*   Код воспроизводим (то есть проверяющий, запустив ноутбук, получит ровно такие же результаты).
*   Все условия задач выполнены.





## Обязательные задачи

### Постановка общей задачи:



Напомним, что вы работаете с небольшой выборкой из коллекции подержанных автомобилей, выставленных на продажу в Соединённых Штатах. На этих данных вам нужно построить модель классификации, определяющую категорию цены подержанного автомобиля в зависимости от характеристик транспортного средства.

### Описание датасета:



- `id`: идентификатор записи;
- `url`: URL записи о продаже;
- `region`: регион;
- `region_url`: URL региона;
- `price`: стоимость;
- `year`: год выпуска;
- `manufacturer`: производитель;
- `model`: модель;
- `fuel`: тип топлива;
- `odometer`: количество пройденных миль;
- `title_status`: статус;
- `transmission`: коробка передач;
- `image_url`: URL изображения;
- `description`: указанное описание;
- `state`: штат;
- `lat`: широта;
- `long`: долгота;
- `posting_date`: дата размещения объявления о продаже;
- `price_category`: категория цены;
- `date`: дата публикации.

###  Задачи:

Ниже представлены задачи по проекту построения модели. Выполните их и в отдельном файле соберите финальный ноутбук проекта.

0. *Импорт сторонних библиотек*

В первой ячейке ноутбука импортируйте библиотеки, которые необходимы для работы с данными. Впоследствии все импорты добавляйте в эту ячейку. Структурируйте их по PEP-8.

[Документация по оформлению import'ов (PEP-8)](https://peps.python.org/pep-0008/#imports)

In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix 
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import RandomizedSearchCV
import pickle

1. *Загрузка данных*

Загрузите данные из файла `df_out`. Выведите размерность и первые 5 строчек данных.

In [1320]:
df_out = pd.read_csv(r'df_out.csv', index_col=0)
print(df_out.shape)
df_out.head()

(10000, 19)


url  \
id                                                                                                             
7308295377   https://chattanooga.craigslist.org/ctd/d/chattanooga-2020-ram-2500-crew-cab-big/7308295377.html   
7316380095               https://newjersey.craigslist.org/ctd/d/carlstadt-2016-ford-explorer/7316380095.html   
7313733749                 https://reno.craigslist.org/ctd/d/atlanta-2017-vw-volkswagen-golf/7313733749.html   
7308210929    https://fayetteville.craigslist.org/ctd/d/raleigh-2013-toyota-rav4-xle-4dr-suv/7308210929.html   
7316474668  https://newyork.craigslist.org/lgi/cto/d/baldwin-2021-nissan-altima-sv-with-only/7316474668.html   

                   region                           region_url  price    year  \
id                                                                              
7308295377    chattanooga   https://chattanooga.craigslist.org  54990  2020.0   
7316380095   north jersey     https://newjersey.craigslist.org  16942  2016.0   
7313733749   reno / tahoe          https://reno.craigslist.org  35590  2017.0   
7308210929   fayetteville  https://fayetteville.craigslist.org  14500  2013.0   
7316474668  new york city       https://newyork.craigslist.org  21800  2021.0   

           manufacturer                   model    fuel  odometer  \
id                                                                  
7308295377          ram  2500 crew cab big horn  diesel     27442   
7316380095         ford    explorer 4wd 4dr xlt     NaN     60023   
7313733749   volkswagen        golf r hatchback     gas     14048   
7308210929       toyota                    rav4     gas    117291   
7316474668       nissan                  altima     gas      8000   

           title_status transmission  \
id                                     
7308295377        clean        other   
7316380095        clean    automatic   
7313733749        clean        other   
7308210929        clean    automatic   
7316474668        clean    automatic   

                                                                      image_url  \
id                                                                                
7308295377  https://images.craigslist.org/00N0N_1xMPvfxRAIdz_0gw0co_600x450.jpg   
7316380095  https://images.craigslist.org/00x0x_26jl9F0cnLvz_0ak06T_600x450.jpg   
7313733749  https://images.craigslist.org/00y0y_eeZjWeiSfbUz_0gw0co_600x450.jpg   
7308210929  https://images.craigslist.org/00606_iGe5iXidibAz_0hd0ew_600x450.jpg   
7316474668  https://images.craigslist.org/00V0V_3pSOiPZ3Sdwz_0x20oM_600x450.jpg   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

# Data Preparation

2. *Data Preparation*

Объявите блок Data Preparation. В этом блоке:
* произведите преобразование типов данных, если нужно;
* исследуйте данные на пропуски, обработайте их (например, заполните какими-то значениями);
* избавьтесь от аномалий, если нужно.

В решении данной задачи опирайтесь на результаты работы в 17-19 модулях блока Data Preparation. Исследование данных подкрепляйте соответствующей визуализацией в виде графиков, таблиц или статистических значений.

Декомпозируйте блок на подблоки.


In [1321]:
df = df_out.copy()

In [1322]:
def count_empty(df):
    for col in df.columns:
        null_qunatity = sum(df[col].isnull())
        if null_qunatity > 0:
            print(col, null_qunatity)

count_empty(df)            

year 36
model 128
fuel 63
title_status 166
transmission 45
image_url 2
description 2
lat 98
long 98
posting_date 2
date 2


In [1323]:
df.dtypes

url                object
region             object
region_url         object
price               int64
year              float64
manufacturer       object
model              object
fuel               object
odometer            int64
title_status       object
transmission       object
image_url          object
description        object
state              object
lat               float64
long              float64
posting_date       object
price_category     object
date               object
dtype: object

## date preparation

In [1324]:
def get_quantile_bound(q1, q3):
    iqr = q3-q1
    bound = (q1 - 1.5 * iqr, q3 + 1.5 * iqr)
    return bound

In [1325]:
df[df.date.isnull()]

,url,region,region_url,price,year,manufacturer,model,fuel,odometer,title_status,transmission,image_url,description,state,lat,long,posting_date,price_category,date
id,,,,,,,,,,,,,,,,,,,
7315259025,https://nh.craigslist.org/cto/d/manchester-2012-chevrolet-impala-lt/7315259025.html,new hampshire,https://nh.craigslist.org,3750,NaN,other,NaN,NaN,95674,NaN,NaN,NaN,NaN,nh,NaN,NaN,NaN,low,NaN
7208549803,https://bellingham.craigslist.org/ctd/d/bellingham-2011-mercedes-benz-class-300/7208549803.html,bellingham,https://bellingham.craigslist.org,11999,NaN,other,NaN,NaN,95674,NaN,NaN,NaN,NaN,wa,NaN,NaN,NaN,medium,NaN


In [1326]:
df.dropna(subset=['date'], inplace=True)
count_empty(df)

year 34
model 126
fuel 61
title_status 164
transmission 43
lat 96
long 96


In [1327]:
df.date = pd.to_datetime(df.date)
df.date.dtype

datetime64[ns, UTC]

In [1328]:
df.date.describe()

count                                   9998
mean     2021-04-24 03:02:46.415083008+00:00
min                2021-04-04 07:32:35+00:00
25%         2021-04-17 13:26:14.500000+00:00
50%         2021-04-26 13:47:57.500000+00:00
75%                2021-05-01 14:18:46+00:00
max                2021-05-05 03:26:24+00:00
Name: date, dtype: object

## long and lat preparation

In [1329]:
df[df.long.isnull()]

url  \
id                                                                                                            
7312692841     https://tricities.craigslist.org/ctd/d/bmw-series-535i-sport-heat-cool-seats/7312692841.html   
7315526339    https://newjersey.craigslist.org/ctd/d/look-what-just-came-in-2014-bmw-series/7315526339.html   
7308641593  https://winstonsalem.craigslist.org/ctd/d/ford-transit-cargo-vans-work-van-high/7308641593.html   
7309282242      https://onslow.craigslist.org/ctd/d/mercedes-benz-class-250-sport-coupe-car/7309282242.html   
7314847134     https://atlanta.craigslist.org/atl/ctd/d/2008-toyota-tundra-57l-low-96k-mile/7314847134.html   
...                                                                                                     ...   
7312710592    https://athensga.craigslist.org/ctd/d/bmw-x5-awd-4x4-navigation-bluetooth-suv/7312710592.html   
7311736404             https://raleigh.craigslist.org/ctd/d/lexus-nx-300-navigation-sunroof/7311736404.html   
7315596665   https://newjersey.craigslist.org/ctd/d/wow-2013-jeep-grand-cherokee-with-miles/7315596665.html   
7308677951     https://myrtlebeach.craigslist.org/ctd/d/ford-f250-4x4-truck-diesel-crew-cab/7308677951.html   
7312622931                    https://cincinnati.craigslist.org/cto/d/2005-diesel-excursion/7312622931.html   

                           region                           region_url  price  \
id                                                                              
7312692841             tri-cities     https://tricities.craigslist.org  17998   
7315526339           north jersey     https://newjersey.craigslist.org  22995   
7308641593          winston-salem  https://winstonsalem.craigslist.org  20998   
7309282242           jacksonville        https://onslow.craigslist.org  10810   
7314847134                atlanta       https://atlanta.craigslist.org  17900   
...                           ...                                  ...    ...   
7312710592                 athens      https://athensga.craigslist.org  14975   
7311736404  raleigh / durham / CH       https://raleigh.craigslist.org  28998   
7315596665           north jersey     https://newjersey.craigslist.org  16495   
7308677951           myrtle beach   https://myrtlebeach.craigslist.org  33647   
7312622931             cincinnati    https://cincinnati.craigslist.org   7999   

              year   manufacturer             model    fuel  odometer  \
id                                                                      
7312692841  2012.0            bmw          5 series     gas     62665   
7315526339  2014.0            bmw          7 series     gas     81085   
7308641593  2016.0           ford     transit cargo     gas    123825   
7309282242  2014.0  mercedes-benz           c-class     gas    124476   
7314847134  2008.0         toyota            tundra     gas     96000   
...            ...            ...               ...     ...       ...   
7312710592  2013.0            bmw                x5     gas     97195   
7311736404  2018.0          lexus            nx 300     gas     19863   
7315596665  2013.0           jeep    grand cherokee   other     90878   
7308677951  2015.0           ford  f-250 super duty  diesel    153524   
7312622931  2005.0           ford    excursion 2005  diesel    210000   

           title_status transmission  \
id                                     
7312692841        clean    automatic   
7315526339        clean    automatic   
7308641593        clean    automatic   
7309282242        clean    automatic   
7314847134        clean    automatic   
...                 ...          ...   
7312710592        clean    automatic   
7311736404        clean    automatic   
7315596665        clean    automatic   
7308677951        clean    automatic   
7312622931        clean    automatic   

                                                                      image_url  \
id                                                  

In [1330]:
df.long = df.long.fillna(df.long.mode()[0])
df.lat = df.lat.fillna(df.lat.mode()[0])

count_empty(df)

year 34
model 126
fuel 61
title_status 164
transmission 43


## transmission preparation

In [1331]:
df.transmission = df.transmission.astype('category')
df.transmission.dtype

CategoricalDtype(categories=['automatic', 'manual', 'other'], ordered=False, categories_dtype=object)

In [1332]:
df[df.transmission.isnull()]

url  \
id                                                                                                              
7305109580   https://grandrapids.craigslist.org/ctd/d/comstock-park-2013-lexus-es-350-4dr-sdn/7305109580.html   
7314951119           https://southjersey.craigslist.org/ctd/d/chesterfield-2012-jeep-wrangler/7314951119.html   
7308021604           https://southjersey.craigslist.org/ctd/d/maple-shade-2015-honda-civic-ex/7308021604.html   
7314781483                 https://nh.craigslist.org/ctd/d/plaistow-2018-ram-ram-chassis-3500/7314781483.html   
7315044723               https://poconos.craigslist.org/ctd/d/chesterfield-2014-jeep-wrangler/7315044723.html   
7311083008       https://greenville.craigslist.org/ctd/d/greenville-2013-volkswagen-jetta-tdi/7311083008.html   
7303942891             https://wenatchee.craigslist.org/ctd/d/everett-2005-honda-pilot-ex-get/7303942891.html   
7310344536           https://roanoke.craigslist.org/ctd/d/roanoke-2007-ford-mustang-v6-deluxe/7310344536.html   
7307857568     https://fortmyers.craigslist.org/lee/ctd/d/fort-myers-2012-jeep-grand-cherokee/7307857568.html   
7314592581                        https://pueblo.craigslist.org/ctd/d/pueblo-2004-ford-ranger/7314592581.html   
7307459834    https://tallahassee.craigslist.org/ctd/d/ocala-2013-ford-450-super-duty-4x2-4dr/7307459834.html   
7314485544        https://stlouis.craigslist.org/ctd/d/saint-peters-2019-nissan-sentra-sv-500/7314485544.html   
7302733118                 https://harrisburg.craigslist.org/ctd/d/brockport-2016-ford-fusion/7302733118.html   
7314386590          https://roanoke.craigslist.org/ctd/d/roanoke-2013-nissan-altima-25-sl-all/7314386590.html   
7307686006          https://cincinnati.craigslist.org/ctd/d/newport-2012-chevrolet-chevy-volt/7307686006.html   
7310989457           https://oklahomacity.craigslist.org/ctd/d/fort-riley-2016-subaru-outback/7310989457.html   
7314143519         https://boise.craigslist.org/ctd/d/nampa-1971-american-lafrance-fire-truck/7314143519.html   
7314284239      https://gainesville.craigslist.org/ctd/d/kissimmee-2012-lexus-is-250-rwd-call/7314284239.html   
7313340095                    https://flint.craigslist.org/ctd/d/burton-2018-nissan-sentra-sv/7313340095.html   
7309751102    https://providence.craigslist.org/ctd/d/hooksett-2015-jeep-wrangler-rubicon-4wd/7309751102.html   
7316685506          https://tampa.craigslist.org/hil/ctd/d/brandon-2016-ford-mustang-ecoboost/7316685506.html   
7307388962                  https://medford.craigslist.org/ctd/d/medford-2010-ford-expedition/7307388962.html   
7315518091   https://inlandempire.craigslist.org/ctd/d/fontana-2012-gmc-yukon-slt-we-have-all/7315518091.html   
7303944212        https://yakima.craigslist.org/ctd/d/everett-2013-dodge-grand-caravan-se-get/7303944212.html   
7302829978           https://greenville.craigslist.org/ctd/d/greenville-2016-jeep-compass-4x4/7302829978.html   
7305971655    https://lancaster.craigslist.org/ctd/d/brockport-2012-jeep-patriot-latitude-4wd/7305971655.html   
7303882218            https://chico.craigslist.org/ctd/d/sacramento-2018-ford-150-f150-150-xl/7303882218.html   
7308278778           https://lincoln.craigslist.org/ctd/d/fort-riley-2013-jeep-grand-cherokee/7308278778.html   
7310860252         https://charlotte.craigslist.org/ctd/d/greensboro-2008-honda-civic-cpe-2dr/7310860252.html   
7312392950         https://stlouis.craigslist.org/ctd/d/saint-peters-2015-ford-explorer-sport/7312392950.html   
7310766348             https://greensboro.craigslist.org/ctd/d/high-point-2001-honda-civic-ex/7310766348.html   
7311954478            https://slo.craigslist.org/ctd/d/templeton-2018-chevrolet-chevy-express/7311954478.html   
7311616552         https://easttexas.craigslist.org/ctd/d/richardson-2015-lexus-is-250-ase30l/7311616552.html   
7307751569          https://ocala.craigslist.org/ctd/d/kissimmee-2020-hyundai-elantra-limited/7307751569.html   
7311607084        https://southjersey.

In [1333]:
df.transmission.describe()

count          9955
unique            3
top       automatic
freq           7700
Name: transmission, dtype: object

In [1334]:
df.transmission.value_counts()

transmission
automatic    7700
other        1624
manual        631
Name: count, dtype: int64

In [1335]:
df.transmission = df.transmission.fillna('other')
count_empty(df)

year 34
model 126
fuel 61
title_status 164


## Title status

In [1336]:
df[df.title_status.isnull()]

url  \
id                                                                                                              
7316540706       https://cincinnati.craigslist.org/ctd/d/fairfield-2012-fiat-500-pop-for-only/7316540706.html   
7313570594  https://orangecounty.craigslist.org/ctd/d/corona-2013-ram-1500-tradesman-quad-cab/7313570594.html   
7315211652         https://ocala.craigslist.org/ctd/d/riverview-2019-toyota-sienna-wheelchair/7315211652.html   
7306702976         https://toledo.craigslist.org/ctd/d/chelsea-73-mo-chevrolet-avalanche-1500/7306702976.html   
7307609203                     https://roseburg.craigslist.org/ctd/d/eugene-2017-ford-mustang/7307609203.html   
...                                                                                                       ...   
7301707830           https://tricities.craigslist.org/ctd/d/bristol-2013-honda-odyssey-5dr-ex/7301707830.html   
7313949726    https://harrisburg.craigslist.org/ctd/d/brockport-2012-honda-accord-lx-sedan-at/7313949726.html   
7314987181        https://nashville.craigslist.org/ctd/d/nashville-2014-lexus-rx-lexus-rx-350/7314987181.html   
7309717260       https://savannah.craigslist.org/ctd/d/jacksonville-2006-lexus-gx-470-4dr-suv/7309717260.html   
7315557731   https://minneapolis.craigslist.org/csw/ctd/d/dassel-2008-ford-f750-750-750-super/7315557731.html   

                           region                           region_url  price  \
id                                                                              
7316540706             cincinnati    https://cincinnati.craigslist.org   4995   
7313570594          orange county  https://orangecounty.craigslist.org  17900   
7315211652                  ocala         https://ocala.craigslist.org  40990   
7306702976                 toledo        https://toledo.craigslist.org   4592   
7307609203               roseburg      https://roseburg.craigslist.org  18450   
...                           ...                                  ...    ...   
7301707830             tri-cities     https://tricities.craigslist.org   8995   
7313949726             harrisburg    https://harrisburg.craigslist.org   8900   
7314987181              nashville     https://nashville.craigslist.org  20900   
7309717260  savannah / hinesville      https://savannah.craigslist.org  11495   
7315557731  minneapolis / st paul   https://minneapolis.craigslist.org  38943   

              year manufacturer                    model    fuel  odometer  \
id                                                                           
7316540706  2012.0         fiat                  500 pop     gas     59242   
7313570594  2013.0          ram  1500 tradesman quad cab     gas    145499   
7315211652  2019.0       toyota               sienna xle     gas      3700   
7306702976  2003.0    chevrolet           avalanche 1500     gas    268691   
7307609203  2017.0         ford                  mustang     gas     66049   
...            ...          ...                      ...     ...       ...   
7301707830  2013.0        honda                  odyssey     gas    203112   
7313949726  2012.0        honda                   accord     NaN     95674   
7314987181  2014.0        lexus                   rx 350     gas     70800   
7309717260  2006.0        lexus                   gx 470     gas    242000   
7315557731  2008.0         ford   f750 super duty - 55ft  diesel     81214   

           title_status transmission  \
id                                     
7316540706          NaN       manual   
7313570594          NaN    automatic   
7315211652          NaN    automatic   
7306702976          NaN    automatic   
7307609203          NaN    automatic   
...                 ...          ...   
7301707830          NaN    automatic   
7313949726          NaN        other   
7314987181          NaN    automatic   
7309717260          NaN    automatic   
7315557731          NaN    automatic   

                                              

In [1337]:
df.title_status.value_counts()

title_status
clean         9518
rebuilt        171
salvage         92
lien            35
missing         16
parts only       2
Name: count, dtype: int64

In [1338]:
df.title_status = df.title_status.astype('category')
df.title_status.dtype

CategoricalDtype(categories=['clean', 'lien', 'missing', 'parts only', 'rebuilt',
                  'salvage'],
, ordered=False, categories_dtype=object)

In [1339]:
df.title_status = df.title_status.fillna(df.title_status.mode()[0])
count_empty(df)

year 34
model 126
fuel 61


## Fuel

In [1340]:
df[df.fuel.isnull()]

,url,region,region_url,price,year,manufacturer,model,fuel,odometer,title_status,transmission,image_url,description,state,lat,long,posting_date,price_category,date
id,,,,,,,,,,,,,,,,,,,
7316380095,https://newjersey.craigslist.org/ctd/d/carlstadt-2016-ford-explorer/7316380095.html,north jersey,https://newjersey.craigslist.org,16942,2016.0,ford,explorer 4wd 4dr xlt,NaN,60023,clean,automatic,https://images.craigslist.org/00x0x_26jl9F0cnLvz_0ak06T_600x450.jpg,"***Call Us for more information at: 201-635-1400*** Visit us online www.projectoneauto.com2016 FORD EXPLORER XLT ECOBOOST 4WD - SHADOW BLACK EXTERIOR / BLACK INTERIOR - 60,023 MILES - BACK UP CAMERA - KEYLESS ENTRY - CRUISE CONTROL - AUXILIARY INPUT - SATELLITE RADIO - ALLOY WHEELS - 2.3L TURBO - ALL WHEEL DRIVE - AUTOMATIC TRANSMISSION - VISIT WWW.PROJECTONEAUTO.COM TO VIEW MORE PICTURES OR CALL 201-635-1400 - WE FINANCE - NO HIDDEN DEALER FEES - WE BUY ALL CARS - LOCATED TEN MINUTES WEST OF MANHATTAN AND ONE MILE NORTH OF METLIFE STADIUM - OPEN MONDAY THROUGH SATURDAY 9 TO 7.Stock #: 13510VIN: 1FM5K8DH5GGA78852Year: 2016Make: FordModel: ExplorerStyle: 4WD 4dr XLTBodyStyle: Sport UtilityExt. Color1: BLACKTransmission: AutomaticDrive Train: Four Wheel DriveCertified: NoMileage: 60023Fuel Type: Flex Fuel CapabilityEngine Cylinders: V6 Cylinder EngineCall for more info: 201-635-1400***Call Us about this 201-635-1400 Visit us online www.projectoneauto.com",nj,40.821805,-74.061962,2021-05-03T15:40:21-0400,medium,2021-05-03 19:40:21+00:00
7309910590,https://eugene.craigslist.org/ctd/d/eugene-2001-toyota-camry-solara/7309910590.html,eugene,https://eugene.craigslist.org,4977,2001.0,toyota,camry solara 2dr cpe se v6 auto (natl),NaN,193242,clean,automatic,https://images.craigslist.org/00F0F_ebtJlxCQjvxz_0ak07K_600x450.jpg,"***Call Us for more information at: 541-228-9437*** Visit us online louglutz.comCheck out this gently-used 2001 Toyota Camry Solara we recently got in. This vehicle is loaded with great features, plus it comes with the CARFAX BuyBack Guarantee. With less than 193,242mi on this Toyota Camry Solara, you'll appreciate the practically showroom newness of this vehicle. Based on the superb condition of this vehicle, along with the options and color, this Toyota Camry Solara SE is sure to sell fast. There is no reason why you shouldn't buy this Toyota Camry Solara SE. It is incomparable for the price and quality.Stock #: L2633VIN: 2T1CF28P21C492336Year: 2001Make: ToyotaModel: Camry SolaraStyle: 2dr Cpe SE V6 Auto (Natl)BodyStyle: 2dr CarExt. Color1: Silver Stream OTransmission: AutomaticDrive Train: Front Wheel DriveCertified: NoMileage: 193242Fuel Type: Gasoline FuelEngine Cylinders: V6 Cylinder EngineCall for more info: 541-228-9437***Call Us about this 541-228-9437 Visit us online louglutz.com",or,44.076513,-123.150156,2021-04-20T10:40:15-0700,low,2021-04-20 17:40:15+00:00
7314559074,https://eugene.craigslist.org/ctd/d/eugene-2009-ford-150/7314559074.html,eugene,https://eugene.craigslist.org,27977,2009.0,ford,"f-150 4wd supercrew 145"" platinum",NaN,79522,clean,automatic,https://images.craigslist.org/00L0L_khoMmqXaSJvz_0ak07K_600x450.jpg,"***Call Us for more information at: 541-228-9437*** Visit us online louglutz.comLooking for a clean, well-cared for 2009 Ford F-150? This is it. This 2009 Ford F-150 comes with a CARFAX Buyback Guarantee, which means you can buy with certainty. With amenities and luxuries befitting a modern day king, this splendorous coach reigns supreme. Tackle any terrain with this 4WD Ford F-150 Platinum. Off the road or on the parkway, you'll drive with confidence no matter the weather conditions or landscape. One of the best things about this Ford F-150 is that it has low, low mileage. It's ready for you to truly break it in. More information about the 2009 Ford F-150: The F-150 has been the best-selling truck nameplate for 31 years, and Ford shows no indication of giving that title up with the completely redesigned model this year. While the new 

In [1341]:
df.fuel.value_counts()

fuel
gas         8402
other        730
diesel       653
hybrid       108
electric      44
Name: count, dtype: int64

In [1342]:
df.fuel = df.fuel.fillna(df.fuel.mode()[0])
count_empty(df)

year 34
model 126


In [1343]:
df.fuel = df.fuel.astype('category')

## Model

In [1344]:
df[df.model.isnull()]

,url,region,region_url,price,year,manufacturer,model,fuel,odometer,title_status,transmission,image_url,description,state,lat,long,posting_date,price_category,date
id,,,,,,,,,,,,,,,,,,,
7316913641,https://ventura.craigslist.org/cto/d/camarillo-2005-range-rover-needs-work/7316913641.html,ventura county,https://ventura.craigslist.org,3000,2005.0,rover,NaN,gas,165206,clean,automatic,https://images.craigslist.org/00S0S_b3yx6L4N2SSz_0jm0ew_600x450.jpg,"2005 Range Rover HSE Changed Oil and now engine has a rattle. Online forums say timing belt tensioner- but who knows. Not running right and must be towed. Was running fine before this, but I am over it. 165k miles Clean title in my name. Current registration New 20” tires (1month old) on Antera rims. (Rims have all center caps but could use a refresh) OEM 19” Land Rover rims with old tires included. $3k OBO. Make an offer. Need this gone. No scams, checks, wires. (Please don’t waste my time)",ca,34.219565,-119.001986,2021-05-04T13:11:15-0700,low,2021-05-04 20:11:15+00:00
7316550853,https://chicago.craigslist.org/chc/cto/d/midlothian-2006-jeep-liberty-4x4-only/7316550853.html,chicago,https://chicago.craigslist.org,3000,2006.0,jeep,NaN,gas,132000,clean,automatic,https://images.craigslist.org/01111_kNv3EKjH4HAz_0CI0ik_600x450.jpg,"SUPER CLEAN 2006 JEEP LIBEERY RANAGADE 4X4 ONLY 132,420 MILES AUTOMATIC TRANSMISSION SHIFTS SMOOTH. .. NO SLIPPING ENGINE RUNS GREAT. .. NO LEAKS NO NOISES NO MISFIRES POWER DOORLOCKS. NO MECHANICAL PROBLEMS AT ALL. EVERYTHING WORKS. LIKE NEW TIRES. . BRAND NEW BRAKE PADS FRONT AND BACK NEW NO ACCIDENTS NO DENTS... CLEAN BODY... NO PAINT FADE CLOTH SEATS IN GREAT SHAPE NO RIPS NO STAINS CLEAN TITLE IN HAND show contact info AKS FOR BRIAN",il,41.618957,-87.726334,2021-05-03T20:33:40-0500,low,2021-05-04 01:33:40+00:00
7303575863,https://worcester.craigslist.org/ctd/d/fitchburg-2007-gmc-with-plow/7303575863.html,worcester / central MA,https://worcester.craigslist.org,7800,2007.0,gmc,NaN,gas,190000,clean,automatic,https://images.craigslist.org/00f0f_iWjiToqea2Zz_1ck0TK_600x450.jpg,2007 gmc with plow leather loaded nice truck runs and drives great call vic or brian at 978-342-8978 $7800,ma,42.579600,-71.803100,2021-04-08T09:10:07-0400,low,2021-04-08 13:10:07+00:00
7315267860,https://gainesville.craigslist.org/ctd/d/kissimmee-2012-ram-1500-laramie-crew/7315267860.html,gainesville,https://gainesville.craigslist.org,19200,2012.0,ram,NaN,gas,121371,clean,automatic,https://images.craigslist.org/00W0W_hHV4D0Po0Gzz_0ak07K_600x450.jpg,"2012 RAM 1500 Laramie Crew Cab 2WD Offered by: WE ARE OPEN EVERYONE APPROVED! 3 DAY 200 MILES EXCHANGE POLICY — (407) 848-1115 — $19,200 Sagebrush Pearlcoat 2012 Ram 1500 Laramie RWD 6-Speed Automatic HEMI 5.7L V8 Multi Displacement VVTOdometer is 7636 miles below market average!Awards: * 2012 KBB.com 10 Best Used Trucks Under $20000First time buyers welcomed! Best program in Central Florida. Bad credit no credit no problem. Everybody rides! Trade-ins welcomed. We have one of the largest inventories over 175 cars to choose from. Best Price is financed with our lender on Tier 1 and EQUIFAX Excellent Credit plus tax title tag documents stamps & dealer fees. Please see the dealer for details. For more information visit WWW.RIKERSINC.COM.Due to market conditions vehicle price maybe adjusted daily. Reviews: * If you need a pickup truck that is as tough as the rugged image it portrays but youd also like something with a civilized ride and a nice interior to double as the family road-trip machine the 2012 Ram 1500 pickup up makes a nice compromise. And the Hemi badge on the side is sure to impress your buddies. Source: KBB.com * Best-in-class ride quality; well-made interior; clever storage spaces; powerful V8; lots of interior space. Source: Edmunds WE ARE OPEN EVERYONE APPROVED! 3 DAY 200 MILES EXCHANGE POLICY Year: 2012 Make: RAM Model: 1500 Series: Laramie Crew Cab 2WD VIN: 1C6RD6NT4CS174350 Stock #: RA13573 Condition: Used Mileage: 121,371 MPG:

In [1345]:
df.model.value_counts()

model
f-150                        151
silverado 1500               110
1500                         100
camry                         77
silverado                     74
                            ... 
grand caravan sport            1
Nascar                         1
titan le                       1
sorrento lx v6                 1
a3 2.0t premium plus pzev      1
Name: count, Length: 3466, dtype: int64

In [1346]:
df.model = df.model.fillna('other')
count_empty(df)
df.shape

year 34


(9998, 19)

## Year

In [1347]:
df[df.year.isnull()]

url  \
id                                                                                                              
7304102133       https://wenatchee.craigslist.org/ctd/d/wenatchee-2019-ram-1500-big-horn-lone/7304102133.html   
7316337069      https://lasvegas.craigslist.org/ctd/d/scottsdale-2020-ford-super-duty-350-srw/7316337069.html   
7315662117     https://westslope.craigslist.org/ctd/d/denver-2019-dodge-grand-caravan-se-gray/7315662117.html   
7316199009          https://nashville.craigslist.org/ctd/d/nashville-2019-chevrolet-silverado/7316199009.html   
7316840794           https://denver.craigslist.org/ctd/d/american-fork-2020-ford-150-platinum/7316840794.html   
7311915616     https://westslope.craigslist.org/ctd/d/denver-2020-honda-odyssey-ex-w-navi-res/7311915616.html   
7306735187     https://westslope.craigslist.org/ctd/d/denver-2019-dodge-grand-caravan-se-blue/7306735187.html   
7309954650    https://gainesville.craigslist.org/ctd/d/gainesville-2019-honda-cr-ex-2wd-lunar/7309954650.html   
7313865045      https://westslope.craigslist.org/ctd/d/denver-2018-toyota-sienna-le-automatic/7313865045.html   
7303344554            https://daytona.craigslist.org/ctd/d/new-smyrna-beach-2020-ford-transit/7303344554.html   
7311762820           https://huntsville.craigslist.org/ctd/d/new-castle-2019-toyota-camry-xse/7311762820.html   
7315662074       https://santafe.craigslist.org/ctd/d/denver-2019-dodge-grand-caravan-se-gray/7315662074.html   
7314588156  https://inlandempire.craigslist.org/ctd/d/hesperia-chevrolet-impala-sunday-public/7314588156.html   
7310250813          https://omaha.craigslist.org/ctd/d/council-bluffs-2019-gmc-acadia-awd-4dr/7310250813.html   
7306678710     https://gainesville.craigslist.org/ctd/d/gainesville-2018-nissan-maxima-sr-35l/7306678710.html   
7316562384   https://honolulu.craigslist.org/oah/ctd/d/honolulu-2018-jeep-grand-cherokee-high/7316562384.html   
7307209305     https://northmiss.craigslist.org/ctd/d/vicksburg-2020-ram-3500-ram-3500-dually/7307209305.html   
7304711487     https://westslope.craigslist.org/ctd/d/denver-2020-honda-odyssey-ex-w-navi-res/7304711487.html   
7309869204   https://scottsbluff.craigslist.org/ctd/d/denver-2019-dodge-grand-caravan-se-plus/7309869204.html   
7304693422      https://shreveport.craigslist.org/ctd/d/vicksburg-2020-gmc-sierra-1500-sierra/7304693422.html   
7310371320           https://westslope.craigslist.org/ctd/d/denver-2018-buick-enclave-fwd-4dr/7310371320.html   
7309904568               https://omaha.craigslist.org/ctd/d/omaha-2020-buick-envision-awd-4dr/7309904568.html   
7315207437     https://huntsville.craigslist.org/ctd/d/new-castle-2020-bmw-series-330i-alpine/7315207437.html   
7310924082          https://omaha.craigslist.org/ctd/d/omaha-2018-subaru-impreza-20i-door-cvt/7310924082.html   
7316247218       https://lasvegas.craigslist.org/ctd/d/scottsdale-2021-ford-150-new-2021-ford/7316247218.html   
7314898718   https://gainesville.craigslist.org/ctd/d/gainesville-2019-ram-2500-tradesman-4x2/7314898718.html   
7314454466       https://billings.craigslist.org/ctd/d/denver-2019-toyota-sienna-le-automatic/7314454466.html   
7310829676        https://tucson.craigslist.org/ctd/d/tempe-2020-gmc-sierra-2500hd-lifted-at4/7310829676.html   
7316852999          https://memphis.craigslist.org/ctd/d/memphis-2019-mazda-cx-sport-fwd-soul/7316852999.html   
7316270709         https://denver.craigslist.org/ctd/d/denver-2019-toyota-sienna-le-automatic/7316270709.html   
7312938642          https://lasvegas.craigslist.org/ctd/d/scottsdale-2020-chevrolet-silverado/7312938642.html   
7309216259                https://omaha.craigslist.org/ctd/d/omaha-2019-buick-enclave-awd-4dr/7309216259.html   
7316246632       https://sanantonio.craigslist.org/ctd/d/scottsdale-2019-ram-1500-lifted-2019/7316246632.html   
7314033574   https://inlandempire.craigslist.org/ctd/d/scottsdale-2020-ford-miles-lifted-ford/7314033574.html   

                             region  

In [1348]:
df = df.dropna(subset=['year'])
count_empty(df)
df.shape

(9964, 19)

In [1349]:
year_bound = get_quantile_bound(df.year.quantile(0.25), df.year.quantile(0.75))
print(year_bound)
df.year.describe()

(1994.5, 2030.5)


count    9964.000000
mean     2010.927941
std         9.672040
min      1915.000000
25%      2008.000000
50%      2013.000000
75%      2017.000000
max      2022.000000
Name: year, dtype: float64

In [1350]:
len(df[df.year<=int(year_bound[0])])

394

In [1351]:
df = df[df.year>int(year_bound[0])]
df.shape

(9570, 19)

## Search for other anomalies

In [1352]:
df.columns

Index(['url', 'region', 'region_url', 'price', 'year', 'manufacturer', 'model',
       'fuel', 'odometer', 'title_status', 'transmission', 'image_url',
       'description', 'state', 'lat', 'long', 'posting_date', 'price_category',
       'date'],
      dtype='object')

In [1353]:
df.price.describe().astype('str')

count                9570.0
mean     20830.087774294672
std      127335.73189610227
min                   500.0
25%                  7950.0
50%                 15988.0
75%                 27995.0
max              12345678.0
Name: price, dtype: object

In [1354]:
price_bound = get_quantile_bound(df.price.quantile(0.25), df.price.quantile(0.75))

In [1355]:
len(df[df.price >= price_bound[1]])

199

In [1356]:
df = df[df.price<=price_bound[1]]
df.shape

(9371, 19)

## Odometer

In [1357]:
df.odometer.describe()

count      9371.000000
mean      95436.203927
std       62696.707377
min           0.000000
25%       40951.000000
50%       91626.000000
75%      138948.000000
max      283466.000000
Name: odometer, dtype: float64

In [1358]:
odometer_bound = get_quantile_bound(df.odometer.quantile(0.25), df.odometer.quantile(0.75))
print(odometer_bound)

(-106044.5, 285943.5)


# Feature engineering

 3. *Feature engineering*

Объявите блок Feature engineering. В этом блоке:
* подготовьте категориальные переменные с помощью OneHotEncoder;
* стандартизируйте и нормализуйте переменные, если нужно;
* создайте новые признаки на основе информации в датафрейме (на основе дат, текстовых значений переменных, и так далее);
* удалите неинформативные колонки, которые появились в датасете в результате Feature engineering;
* сформируйте финальный датасет, на котором будет производиться моделирование, и сохраните его в отдельный файл.

В решении данной задачи опирайтесь на результаты работы в 20 модуле блока Data Preparation. Исследование подкрепляйте соответствующей визуализацией в виде графиков, таблиц или матриц корреляций, если это нужно.

Декомпозируйте блок на подблоки.

In [1359]:
df

url  \
id                                                                                                             
7308295377   https://chattanooga.craigslist.org/ctd/d/chattanooga-2020-ram-2500-crew-cab-big/7308295377.html   
7316380095               https://newjersey.craigslist.org/ctd/d/carlstadt-2016-ford-explorer/7316380095.html   
7313733749                 https://reno.craigslist.org/ctd/d/atlanta-2017-vw-volkswagen-golf/7313733749.html   
7308210929    https://fayetteville.craigslist.org/ctd/d/raleigh-2013-toyota-rav4-xle-4dr-suv/7308210929.html   
7316474668  https://newyork.craigslist.org/lgi/cto/d/baldwin-2021-nissan-altima-sv-with-only/7316474668.html   
...                                                                                                      ...   
7304876387                  https://chautauqua.craigslist.org/ctd/d/falconer-2002-toyota-rav/7304876387.html   
7316152972              https://binghamton.craigslist.org/ctd/d/rochester-2008-jeep-wrangler/7316152972.html   
7310993818       https://salem.craigslist.org/ctd/d/salem-2011-audi-a3-20t-premium-plus-pzev/7310993818.html   
7306637427       https://madison.craigslist.org/ctd/d/madison-2015-porsche-cayenne-awd-4dr-e/7306637427.html   
7311960763        https://norfolk.craigslist.org/ctd/d/chesapeake-2017-ram-1500-crew-cab-big/7311960763.html   

                             region                           region_url  \
id                                                                         
7308295377              chattanooga   https://chattanooga.craigslist.org   
7316380095             north jersey     https://newjersey.craigslist.org   
7313733749             reno / tahoe          https://reno.craigslist.org   
7308210929             fayetteville  https://fayetteville.craigslist.org   
7316474668            new york city       https://newyork.craigslist.org   
...                             ...                                  ...   
7304876387               chautauqua    https://chautauqua.craigslist.org   
7316152972               binghamton    https://binghamton.craigslist.org   
7310993818                    salem         https://salem.craigslist.org   
7306637427                  madison       https://madison.craigslist.org   
7311960763  norfolk / hampton roads       https://norfolk.craigslist.org   

            price    year manufacturer                      model    fuel  \
id                                                                          
7308295377  54990  2020.0          ram     2500 crew cab big horn  diesel   
7316380095  16942  2016.0         ford       explorer 4wd 4dr xlt     gas   
7313733749  35590  2017.0   volkswagen           golf r hatchback     gas   
7308210929  14500  2013.0       toyota                       rav4     gas   
7316474668  21800  2021.0       nissan                     altima     gas   
...           ...     ...          ...                        ...     ...   
7304876387   4495  2002.0       toyota                       rav4     gas   
7316152972  14495  2008.0         jeep                   wrangler     gas   
7310993818   8995  2011.0         audi  a3 2.0t premium plus pzev     gas   
7306637427  31900  2015.0      porsche                    cayenne  hybrid   
7311960763  31990  2017.0          ram     1500 crew cab big horn     gas   

            odometer title_status transmission  \
id                                               
7308295377     27442        clean        other   
7316380095     60023        clean    automatic   
7313733749     14048        clean        other   
7308210929    117291        clean    automatic   
7316474668      8000        clean    automatic   
...              ...          ...          ...   
7304876387    150000        clean    automatic   
7316152972    113573        clean        other   
7310993818    150184        clean    automatic   
7306637427     61943        clean    automatic   
7311960763     35921        clean        other   

      

In [1360]:
new_df = df.drop(['url', 'region_url', 'image_url', 'description'], axis=1)
new_df.shape

(9371, 15)

## Change datatype

In [1361]:
new_df.dtypes

region                         object
price                           int64
year                          float64
manufacturer                   object
model                          object
fuel                         category
odometer                        int64
title_status                 category
transmission                 category
state                          object
lat                           float64
long                          float64
posting_date                   object
price_category                 object
date              datetime64[ns, UTC]
dtype: object

In [1362]:
new_df['posting_date'] = df['posting_date'].apply(lambda x: x[:19])
new_df.posting_date

id
7308295377    2021-04-17T12:30:50
7316380095    2021-05-03T15:40:21
7313733749    2021-04-28T03:52:20
7308210929    2021-04-17T10:08:57
7316474668    2021-05-03T18:32:06
                     ...         
7304876387    2021-04-10T16:33:57
7316152972    2021-05-03T09:36:30
7310993818    2021-04-22T12:14:01
7306637427    2021-04-14T09:14:42
7311960763    2021-04-24T13:50:49
Name: posting_date, Length: 9371, dtype: object

In [1363]:
new_df.region = new_df.region.astype('category')
new_df.manufacturer = new_df.manufacturer.astype('category')
new_df.model = new_df.model.astype('category')

new_df['posting_date'] = pd.to_datetime(new_df['posting_date'], 
                                        format='%Y-%m-%dT%H:%M:%S',
                                        errors='coerce',
                                        utc=True)

new_df.dtypes

region                       category
price                           int64
year                          float64
manufacturer                 category
model                        category
fuel                         category
odometer                        int64
title_status                 category
transmission                 category
state                          object
lat                           float64
long                          float64
posting_date      datetime64[ns, UTC]
price_category                 object
date              datetime64[ns, UTC]
dtype: object

In [1364]:
new_df.head()

,region,price,year,manufacturer,model,fuel,odometer,title_status,transmission,state,lat,long,posting_date,price_category,date
id,,,,,,,,,,,,,,,
7308295377,chattanooga,54990,2020.0,ram,2500 crew cab big horn,diesel,27442,clean,other,tn,35.060000,-85.250000,2021-04-17 12:30:50+00:00,high,2021-04-17 16:30:50+00:00
7316380095,north jersey,16942,2016.0,ford,explorer 4wd 4dr xlt,gas,60023,clean,automatic,nj,40.821805,-74.061962,2021-05-03 15:40:21+00:00,medium,2021-05-03 19:40:21+00:00
7313733749,reno / tahoe,35590,2017.0,volkswagen,golf r hatchback,gas,14048,clean,other,ca,33.779214,-84.411811,2021-04-28 03:52:20+00:00,high,2021-04-28 10:52:20+00:00
7308210929,fayetteville,14500,2013.0,toyota,rav4,gas,117291,clean,automatic,nc,35.715954,-78.655304,2021-04-17 10:08:57+00:00,medium,2021-04-17 14:08:57+00:00
7316474668,new york city,21800,2021.0,nissan,altima,gas,8000,clean,automatic,ny,40.654800,-73.609700,2021-05-03 18:32:06+00:00,medium,2021-05-03 22:32:06+00:00


## date

In [1365]:
max(new_df['date']-new_df['posting_date'])

Timedelta('0 days 10:00:00')

In [1366]:
new_df['hours_in_sale'] = (new_df['date']-new_df['posting_date']).apply(lambda x: x.seconds//3600)
new_df['posting_month'] = new_df['posting_date'].apply(lambda x: x.month)
new_df = new_df.drop(['date', 'posting_date'], axis=1)
new_df.head()

,region,price,year,manufacturer,model,fuel,odometer,title_status,transmission,state,lat,long,price_category,hours_in_sale,posting_month
id,,,,,,,,,,,,,,,
7308295377,chattanooga,54990,2020.0,ram,2500 crew cab big horn,diesel,27442,clean,other,tn,35.060000,-85.250000,high,4,4
7316380095,north jersey,16942,2016.0,ford,explorer 4wd 4dr xlt,gas,60023,clean,automatic,nj,40.821805,-74.061962,medium,4,5
7313733749,reno / tahoe,35590,2017.0,volkswagen,golf r hatchback,gas,14048,clean,other,ca,33.779214,-84.411811,high,7,4
7308210929,fayetteville,14500,2013.0,toyota,rav4,gas,117291,clean,automatic,nc,35.715954,-78.655304,medium,4,4
7316474668,new york city,21800,2021.0,nissan,altima,gas,8000,clean,automatic,ny,40.654800,-73.609700,medium,4,5


## Model

In [1367]:
new_df.model.value_counts()

model
f-150                                147
silverado 1500                       107
1500                                  99
other                                 79
camry                                 77
                                    ... 
f-150 limited 4x4                      1
f-150 platinum crew cab 4x4            1
f-150 platinum crew ecoboost 3.5l      1
f-150 quad cab short box               1
♿                                      1
Name: count, Length: 3156, dtype: int64

In [1368]:
new_df.model = new_df.model.apply(lambda x: x.split(' ')[0])

In [1369]:
new_df.model.value_counts()

model
silverado    384
1500         205
wrangler     201
grand        200
f-150        198
            ... 
jimmy          1
hiace          1
g8             1
s70            1
durangon       1
Name: count, Length: 823, dtype: int64

## Data normalize

In [1370]:
new_df.dtypes

region            category
price                int64
year               float64
manufacturer      category
model               object
fuel              category
odometer             int64
title_status      category
transmission      category
state               object
lat                float64
long               float64
price_category      object
hours_in_sale        int64
posting_month        int64
dtype: object

In [1371]:
from sklearn.preprocessing import StandardScaler

number_columns = ['year', 'odometer', 'lat', 'long', 'hours_in_sale', 'posting_month']
numbers_date = new_df[number_columns]

std = StandardScaler()
std.fit(numbers_date)
std_data = std.transform(numbers_date)

In [1372]:
std_names = [i+'_std' for i in number_columns]
new_df[std_names] = std_data
new_df.drop(number_columns, axis=1, inplace=True)
new_df.head()

,region,price,manufacturer,model,fuel,title_status,transmission,state,price_category,year_std,odometer_std,lat_std,long_std,hours_in_sale_std,posting_month_std
id,,,,,,,,,,,,,,,
7308295377,chattanooga,54990,ram,2500,diesel,clean,other,tn,high,1.391646,-1.084552,-0.627107,0.476400,-0.855040,-0.593660
7316380095,north jersey,16942,ford,explorer,gas,clean,automatic,nj,medium,0.669357,-0.564864,0.382918,1.106553,-0.855040,1.684467
7313733749,reno / tahoe,35590,volkswagen,golf,gas,clean,other,ca,high,0.849929,-1.298195,-0.851625,0.523610,1.533211,-0.593660
7308210929,fayetteville,14500,toyota,rav4,gas,clean,automatic,nc,medium,0.127640,0.348598,-0.512121,0.847838,-0.855040,-0.593660
7316474668,new york city,21800,nissan,altima,gas,clean,automatic,ny,medium,1.572219,-1.394664,0.353643,1.132026,-0.855040,1.684467


## Create ohe

In [1373]:
new_df.head()

,region,price,manufacturer,model,fuel,title_status,transmission,state,price_category,year_std,odometer_std,lat_std,long_std,hours_in_sale_std,posting_month_std
id,,,,,,,,,,,,,,,
7308295377,chattanooga,54990,ram,2500,diesel,clean,other,tn,high,1.391646,-1.084552,-0.627107,0.476400,-0.855040,-0.593660
7316380095,north jersey,16942,ford,explorer,gas,clean,automatic,nj,medium,0.669357,-0.564864,0.382918,1.106553,-0.855040,1.684467
7313733749,reno / tahoe,35590,volkswagen,golf,gas,clean,other,ca,high,0.849929,-1.298195,-0.851625,0.523610,1.533211,-0.593660
7308210929,fayetteville,14500,toyota,rav4,gas,clean,automatic,nc,medium,0.127640,0.348598,-0.512121,0.847838,-0.855040,-0.593660
7316474668,new york city,21800,nissan,altima,gas,clean,automatic,ny,medium,1.572219,-1.394664,0.353643,1.132026,-0.855040,1.684467


In [1374]:
category_df = new_df[['region', 'manufacturer', 'model', 'fuel', 'title_status', 'transmission', 'state']]
ohe = OneHotEncoder(sparse_output=False)
ohe.fit(category_df);

In [1375]:
ohe_data = ohe.transform(category_df)

In [1376]:
new_df[ohe.get_feature_names_out()] = ohe_data
new_df.shape

C:\Users\kuza-\AppData\Local\Temp\ipykernel_73144\3723909882.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df[ohe.get_feature_names_out()] = ohe_data
C:\Users\kuza-\AppData\Local\Temp\ipykernel_73144\3723909882.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  new_df[ohe.get_feature_names_out()] = ohe_data
C:\Users\kuza-\AppData\Local\Temp\ipykernel_73144\3723909882.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  

(9371, 1333)

In [1377]:
new_df = new_df.drop(category_df.columns, axis=1)
new_df.shape

(9371, 1326)

In [1378]:
with open('df_prepared.pkl', 'wb') as f:
    pickle.dump(new_df, f)

# Modelling

4. *Modelling*

Объявите блок Modelling. В этом блоке:
* сформируйте датасет для обучения; инициализируйте фичи и целевую переменную
(`price_category`); положите их в соответствующие переменные;
* разделите данные на треин и тест;
* объявите три модели: логистическая регрессия, случайный лес и многослойный персептрон;
* поработайте с моделями: сделайте тюнинг параметров и выберите лучшую модель с помощью кросс-валидации на тренировочной выборке;
* по результатам кросс-валидации выберите лучшую модель;
* посчитайте значение метрики лучшей модели на тестовой выборке; убедитесь, что модель не переобучилась.

В решении данной задачи опирайтесь на результаты работы в 22-23 модулях блока Modelling. Исследование подкрепляйте соответствующей визуализацией в виде графиков, таблиц или матриц корреляций, если это нужно.

Декомпозируйте блок на подблоки, если это нужно.

In [3]:
with open('df_prepared.pkl', 'rb') as f:
    dataset = pickle.load(f)
dataset.drop(['price'], axis=1, inplace=True)

In [3]:
dataset.head()

,price_category,year_std,odometer_std,lat_std,long_std,hours_in_sale_std,posting_month_std,region_SF bay area,region_abilene,region_akron / canton,...,state_sd,state_tn,state_tx,state_ut,state_va,state_vt,state_wa,state_wi,state_wv,state_wy
id,,,,,,,,,,,,,,,,,,,,,
7308295377,high,1.391646,-1.084552,-0.627107,0.476400,-0.855040,-0.593660,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7316380095,medium,0.669357,-0.564864,0.382918,1.106553,-0.855040,1.684467,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7313733749,high,0.849929,-1.298195,-0.851625,0.523610,1.533211,-0.593660,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7308210929,medium,0.127640,0.348598,-0.512121,0.847838,-0.855040,-0.593660,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7316474668,medium,1.572219,-1.394664,0.353643,1.132026,-0.855040,1.684467,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
x = dataset.drop(['price_category'], axis=1)
y = dataset.price_category

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [5]:
x.head()

,year_std,odometer_std,lat_std,long_std,hours_in_sale_std,posting_month_std,region_SF bay area,region_abilene,region_akron / canton,region_albany,...,state_sd,state_tn,state_tx,state_ut,state_va,state_vt,state_wa,state_wi,state_wv,state_wy
id,,,,,,,,,,,,,,,,,,,,,
7308295377,1.391646,-1.084552,-0.627107,0.476400,-0.855040,-0.593660,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7316380095,0.669357,-0.564864,0.382918,1.106553,-0.855040,1.684467,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7313733749,0.849929,-1.298195,-0.851625,0.523610,1.533211,-0.593660,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7308210929,0.127640,0.348598,-0.512121,0.847838,-0.855040,-0.593660,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7316474668,1.572219,-1.394664,0.353643,1.132026,-0.855040,1.684467,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Logistic Regression

In [11]:
def test_model(model, x_test, y_test):
    print(accuracy_score(y_test, model.predict(x_test)))
    print(confusion_matrix(y_test, model.predict(x_test)))

In [7]:
lr_model = LogisticRegression()

param_grid = {
    'solver': ['newton-cg', 'sag', 'saga', 'lbfgs'],
    'C': [0.5 * i for i in range(5)],
    'class_weight': [None, 'balanced'],
    'max_iter': list(range(1000, 5000, 500)),
}
randomized_search_rf = RandomizedSearchCV(lr_model, param_grid, scoring='accuracy', n_jobs=-1)

randomized_search_rf.fit(x_train, y_train)

best_params = randomized_search_rf.best_params_
print(best_params)

C:\Users\kuza-\miniconda3\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
5 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\kuza-\miniconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\kuza-\miniconda3\Lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "C:\Users\kuza-\miniconda3\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "C:\Users\kuza-\miniconda3\Lib\site-packag

{'solver': 'lbfgs', 'max_iter': 2000, 'class_weight': 'balanced', 'C': 1.0}


In [1311]:
# best_params = {'solver': 'lbfgs', 'max_iter': 2000, 'class_weight': 'balanced', 'C': 1.0}

In [8]:
lr_model = LogisticRegression(**best_params)
lr_model.fit(x_train, y_train)

test_model(lr_model, x_test, y_test)

0.792
[[562   5  87]
 [  7 495  99]
 [ 90 102 428]]


## Randrom Forest

In [9]:
rf_model = RandomForestClassifier()

param_grid = {
   'bootstrap': [True, False],
   'max_depth': [None] + list(range(500, 1500, 100)),
   'min_samples_split': list(range(2, 50)),
   'n_estimators': list(range(500, 1500, 100)),
}
randomized_search_rf = RandomizedSearchCV(
   estimator=rf_model,
   param_distributions=param_grid,
   n_iter=50,
   scoring='accuracy',
   verbose=1,
   n_jobs=-1
)

randomized_search_rf.fit(x_train, y_train)

best_params = randomized_search_rf.best_params_
print(best_params)

Fitting 5 folds for each of 50 candidates, totalling 250 fits
{'n_estimators': 600, 'min_samples_split': 2, 'max_depth': 1300, 'bootstrap': False}


In [1314]:
# best_params = {'n_estimators': 600, 'min_samples_split': 2, 'max_depth': 1300, 'bootstrap': False}

In [10]:
rf_model = RandomForestClassifier()
rf_model.fit(x_train, y_train)

test_model(rf_model, x_test, y_test)

0.7856
[[560  10  84]
 [  6 498  97]
 [102 103 415]]


## MLP

In [6]:
MLP_model = MLPClassifier()

param_grid = {
    'max_iter': list(range(200, 2000, 100)),
    'hidden_layer_sizes': [(500, 400, 300, 200, 100), 
                            (400, 400, 400, 400, 400), 
                            (300, 300, 300, 300, 300), 
                            (200, 200, 200, 200, 200)],
    'activation': ['logistic', 'tanh', 'relu'],
    'alpha': [0.0001, 0.001, 0.005],
    'early_stopping': [True, False]
    }

randomized_search_rf = RandomizedSearchCV(
   estimator=MLP_model,
   param_distributions=param_grid,
   n_iter=10,
   scoring='accuracy',
   verbose=1,
   n_jobs=-17
)

randomized_search_rf.fit(x_train, y_train)

best_params = randomized_search_rf.best_params_
print(best_params)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
{'max_iter': 1200, 'hidden_layer_sizes': (200, 200, 200, 200, 200), 'early_stopping': True, 'alpha': 0.001, 'activation': 'logistic'}


In [ ]:
best_params = {'max_iter': 1200, 'hidden_layer_sizes': (200, 200, 200, 200, 200), 'early_stopping': True, 'alpha': 0.001, 'activation': 'logistic'}

In [12]:
mlp_model = MLPClassifier(**best_params)
mlp_model.fit(x_train, y_train)

test_model(mlp_model, x_test, y_test)

0.8128
[[570   2  82]
 [  6 521  74]
 [ 87 100 433]]


4. *Results*

Объявите блок Results. В этом блоке:

* подведите итог: какая модель показала себя лучше всего и будет финальным результатом данного ноутбука;
* обучите эту модель на всём датасете;
* сохраните обученную модель в `pickle`.

## Results

Лучше всего справился персептрон

In [13]:
mlp_model.fit(x, y)

MLPClassifier(activation='logistic', alpha=0.001, early_stopping=True,
              hidden_layer_sizes=(200, 200, 200, 200, 200), max_iter=1200)

In [14]:
mlp_model.score(x, y)

0.8619144168178423

In [15]:
with open('mlp_model.pkl', 'wb') as f:
    pickle.dump(mlp_model, f)

## Как отправить работу на проверку


Отправьте куратору на проверку финальный ноутбук проекта, а также файл обученной модели в формате `pickle`.